# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 13 2023 10:39AM,255002,19,ADV80010273,"AdvaGen Pharma, Ltd",Released
1,Jan 13 2023 10:07AM,254999,10,9023067,Eywa Pharma Inc.,Released
2,Jan 13 2023 10:07AM,254999,10,9014398-BO,Eywa Pharma Inc.,Released
3,Jan 13 2023 10:07AM,254999,10,9024349,Eywa Pharma Inc.,Released
4,Jan 13 2023 10:07AM,254999,10,9024350,Eywa Pharma Inc.,Released
5,Jan 13 2023 10:07AM,254999,10,9026905,Eywa Pharma Inc.,Released
6,Jan 13 2023 10:07AM,254999,10,9026927,Eywa Pharma Inc.,Released
7,Jan 13 2023 10:01AM,254996,15,VT80182738,"Virtus Pharmaceuticals, LLC",Released
8,Jan 13 2023 10:01AM,254996,15,VT80182739,"Virtus Pharmaceuticals, LLC",Released
9,Jan 13 2023 10:01AM,254996,15,VT80182740,"Virtus Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
20,254995,Released,1
21,254996,Released,5
22,254997,Released,21
23,254999,Released,6
24,255002,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
254995,NaN,1.0
254996,NaN,5.0
254997,NaN,21.0
254999,NaN,6.0
255002,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
254918,0.0,1.0
254922,1.0,0.0
254949,0.0,4.0
254961,1.0,0.0
254976,6.0,42.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
254918,0,1
254922,1,0
254949,0,4
254961,1,0
254976,6,42


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,254918,0,1
1,254922,1,0
2,254949,0,4
3,254961,1,0
4,254976,6,42


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,254918,,1
1,254922,1,
2,254949,,4
3,254961,1,
4,254976,6,42


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 13 2023 10:39AM,255002,19,"AdvaGen Pharma, Ltd"
1,Jan 13 2023 10:07AM,254999,10,Eywa Pharma Inc.
7,Jan 13 2023 10:01AM,254996,15,"Virtus Pharmaceuticals, LLC"
12,Jan 13 2023 10:01AM,254995,15,"Mizner Bioscience, LLC"
13,Jan 13 2023 9:45AM,254997,10,Bio-PRF
34,Jan 13 2023 9:44AM,254994,15,"Person & Covey, Inc."
103,Jan 13 2023 9:44AM,254992,15,"Alliance Pharma, Inc."
111,Jan 13 2023 9:37AM,254993,10,"Citieffe, Inc."
112,Jan 13 2023 9:31AM,254991,10,"Citieffe, Inc."
116,Jan 13 2023 9:26AM,254990,10,Yusen – 3D Matrix


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jan 13 2023 10:39AM,255002,19,"AdvaGen Pharma, Ltd",,1
1,Jan 13 2023 10:07AM,254999,10,Eywa Pharma Inc.,,6
2,Jan 13 2023 10:01AM,254996,15,"Virtus Pharmaceuticals, LLC",,5
3,Jan 13 2023 10:01AM,254995,15,"Mizner Bioscience, LLC",,1
4,Jan 13 2023 9:45AM,254997,10,Bio-PRF,,21
5,Jan 13 2023 9:44AM,254994,15,"Person & Covey, Inc.",24,45
6,Jan 13 2023 9:44AM,254992,15,"Alliance Pharma, Inc.",,8
7,Jan 13 2023 9:37AM,254993,10,"Citieffe, Inc.",,1
8,Jan 13 2023 9:31AM,254991,10,"Citieffe, Inc.",,4
9,Jan 13 2023 9:26AM,254990,10,Yusen – 3D Matrix,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 13 2023 10:39AM,255002,19,"AdvaGen Pharma, Ltd",1,
1,Jan 13 2023 10:07AM,254999,10,Eywa Pharma Inc.,6,
2,Jan 13 2023 10:01AM,254996,15,"Virtus Pharmaceuticals, LLC",5,
3,Jan 13 2023 10:01AM,254995,15,"Mizner Bioscience, LLC",1,
4,Jan 13 2023 9:45AM,254997,10,Bio-PRF,21,
5,Jan 13 2023 9:44AM,254994,15,"Person & Covey, Inc.",45,24
6,Jan 13 2023 9:44AM,254992,15,"Alliance Pharma, Inc.",8,
7,Jan 13 2023 9:37AM,254993,10,"Citieffe, Inc.",1,
8,Jan 13 2023 9:31AM,254991,10,"Citieffe, Inc.",4,
9,Jan 13 2023 9:26AM,254990,10,Yusen – 3D Matrix,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 13 2023 10:39AM,255002,19,"AdvaGen Pharma, Ltd",1,
1,Jan 13 2023 10:07AM,254999,10,Eywa Pharma Inc.,6,
2,Jan 13 2023 10:01AM,254996,15,"Virtus Pharmaceuticals, LLC",5,
3,Jan 13 2023 10:01AM,254995,15,"Mizner Bioscience, LLC",1,
4,Jan 13 2023 9:45AM,254997,10,Bio-PRF,21,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jan 13 2023 10:39AM,255002,19,"AdvaGen Pharma, Ltd",1.0,NaN
1,Jan 13 2023 10:07AM,254999,10,Eywa Pharma Inc.,6.0,NaN
2,Jan 13 2023 10:01AM,254996,15,"Virtus Pharmaceuticals, LLC",5.0,NaN
3,Jan 13 2023 10:01AM,254995,15,"Mizner Bioscience, LLC",1.0,NaN
4,Jan 13 2023 9:45AM,254997,10,Bio-PRF,21.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 13 2023 10:39AM,255002,19,"AdvaGen Pharma, Ltd",1.0,0.0
1,Jan 13 2023 10:07AM,254999,10,Eywa Pharma Inc.,6.0,0.0
2,Jan 13 2023 10:01AM,254996,15,"Virtus Pharmaceuticals, LLC",5.0,0.0
3,Jan 13 2023 10:01AM,254995,15,"Mizner Bioscience, LLC",1.0,0.0
4,Jan 13 2023 9:45AM,254997,10,Bio-PRF,21.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3059832,84.0,28.0
12,254961,0.0,1.0
15,1019977,59.0,24.0
19,509978,43.0,6.0
21,509840,1.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3059832,84.0,28.0
1,12,254961,0.0,1.0
2,15,1019977,59.0,24.0
3,19,509978,43.0,6.0
4,21,509840,1.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,84.0,28.0
1,12,0.0,1.0
2,15,59.0,24.0
3,19,43.0,6.0
4,21,1.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,84.0
1,12,Released,0.0
2,15,Released,59.0
3,19,Released,43.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,21
Status,,,,,
Executing,28.0,1.0,24.0,6.0,1.0
Released,84.0,0.0,59.0,43.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,21
0,Executing,28.0,1.0,24.0,6.0,1.0
1,Released,84.0,0.0,59.0,43.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,21
0,Executing,28.0,1.0,24.0,6.0,1.0
1,Released,84.0,0.0,59.0,43.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()